In [1]:
#%%writefile summarizer.py

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Okt, Kkma
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
np.random.seed(seed=0)

class news_summarizing():
    def __init__(self, news_list):
        df = news_list[0]
        df['분야'] = '정치'
        secs = ['정치','경제', '사회', '생활', '세계', '과학']
        for i in range(1,len(news_list)):
            now = news_list[i]
            now['분야'] = secs[i]
            df = pd.concat([df,now], axis = 0)
            
        self.df = df
        self.df1 = df[df['contents'].str.len() < 200]
        self.df2 = df[df['contents'].str.len() >= 200]
    
    def text2sentence (self,text) : # 기사 한 문단을 하나의 문장 리스트로
        kkma = Kkma()
        sentence = kkma.sentences(text)
        for i in range(len(sentence)):
            if len(sentence[i]) <= 10:
                sentence[i-1] +=(' '+sentence[i])
                sentence[i] = ''
        return sentence
    
    def give_df(self):
        print('----------뉴스 데이터를 처리 중입니다----------')
        self.df2['s1'] = self.df2['contents'].apply(lambda x: self.text2sentence(x))
        return self.df1,self.df2
    

class smz() :
    tfd = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []
    

    def cleansing(text):
        okt =Okt()
        stopwords = ['머니투데이', '연합뉴스', '데일리', '동아일보', '중앙일보',
                    '조선일보', '기자']

        nouns = []
        for sentence in text :
            if sentence is not '':
                nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                       if noun not in stopwords and len(noun) > 1]))
        return nouns


    def mk_sentGraph(x):
        tfd_mat = smz.tfd.fit_transform(x).toarray()
        gs = np.dot(tfd_mat, tfd_mat.T)
        return gs

    def mk_wordsGraph(x):
        cnt_vec_mat = normalize(smz.cnt_vec.fit_transform(x).toarray().astype(float), axis =0)
        voca = smz.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {voca[word] : word for word in voca}

    def get_ranks(graph, d= 0.85) :
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id,id] = 0
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:,id] /= link_sum
            A[:,id] *= -d
            A[id,id] = 1
        B = (1-d) * np.ones((matrix_size,1))
        ranks = np.linalg.solve(A,B)
        return {idx: r[0] for idx, r in enumerate(ranks)}

    def summarize(sentences,ranked):
        sent_num =3
        summary = []
        index = []
        for idx in ranked[:sent_num] :
            index.append(idx)
        index.sort()

        for idx in index:
            summary.append(sentences[idx])
        return ' '.join(summary)
            
    def run(x) : 
        #print('----------뉴스 기사를 요약 중입니다----------')
        cleaned = smz.cleansing(x)

        try :
            sent_graph = smz.mk_sentGraph(cleaned)

            words_graph, idx2word = smz.mk_wordsGraph(cleaned)

            sent_rank_idx = smz.get_ranks(sent_graph)
            word_rank_idx = smz.get_ranks(words_graph)

            sorted_sent_rank_idx = sorted(sent_rank_idx,
                                  key = lambda x : sent_rank_idx[x],
                                          reverse = True)
            sorted_word_rank_idx = sorted(word_rank_idx,
                                  key = lambda x : word_rank_idx[x],
                                          reverse = True)

            result = smz.summarize(x,sorted_sent_rank_idx)
            return result
        except :
            return ''
        
    def concatDF(df1, df2):
        df1['요약문'] = ''
        df_final = pd.concat([df1,df2], axis =0).sort_values(by=['분야'])
        df_final.loc[df_final.요약문 == '', '요약문'] = df_final.loc[df_final.요약문 == '', 'title']
        return df_final.drop(['s1'], axis=1).reset_index(drop = True)

In [2]:
import scraper 

scraps = scraper.news_scraper()
news_list = scraps.start()

----------뉴스 기사 스크랩을 실행합니다----------

정치 뉴스 기사 크롤링 완료
경제 뉴스 기사 크롤링 완료
사회 뉴스 기사 크롤링 완료
생활 뉴스 기사 크롤링 완료
세계 뉴스 기사 크롤링 완료
과학 뉴스 기사 크롤링 완료


## -----------------------------

In [3]:
a= news_summarizing(news_list)
df1, df2 = a.give_df()

----------뉴스 데이터를 처리 중입니다----------


c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
df2

,title,contents,분야,s1
0,"국민의힘 정책 알릴 플랫폼, 네이버도 카카오도 페이스북도 아닌, ‘나무위키’",“앞으로 모든 정책과 공약은 나무위키 통해서 할 것” 국민의힘이 내년 ...,정치,[“ 앞으로 모든 정책과 공약은 나무 위 키 통해서 할 것” 국민의 힘이 내년 대선...
1,"하면 된다?…2030 지지율 역전 李, 하루걸러 하루 ‘청년표’ 노크",이재명 더불어민주당 대선 후보가 7일 오후 서울 마포구 한 모임 공간(앤드스페이...,정치,[이재명 더불어 민주당 대선 후보가 7일 오후 서울 마포구 한 모임 공간( 앤드 스...
2,"윤석열 ""집권하면 북한 피격 공무원 수사정보 공개하겠다""","윤석열 ""집권하면 북한 피격 공무원 수사정보 공개하겠다"" 사진=연합뉴스국민의힘 윤석...",정치,"[윤석 열 "" 집권하면 북한 피격 공무원 수사정보 공개하겠다"" 사진= 연합 뉴스 국..."
3,"文대통령, 12일 호주 국빈방문…청와대 “방역 만전 기해”",문재인 대통령이 지난 10월 31일(현지시간) 이탈리아 로마 누볼라 컨벤션 양자회담...,정치,[문 재인 대통령이 지난 10월 31일( 현지시간) 이탈리아 로마 누 볼라 컨벤션 ...
4,"금태섭 의원에 묻고 싶다, 국민의힘은 뭐가 바뀌었나","""주 52시간 폐지? 윤석열, 노동관 바꾸지 않으면 사퇴해야"" http://omn...",정치,"["" 주 52 시간 폐지? 윤석 열, 노동 관 바꾸지 않으면 사퇴해야"" http: ..."
...,...,...,...,...
11,“2022년 화두는 알츠하이머병·유전자치료제…신약개발사 관심”,투자 토크쇼바이오 기업에 투자하려면 무엇을 알아야 할까. 어떤 바이오 기업에 투자...,과학,"[투자 토크쇼 바이오 기업에 투자하려면 무엇을 알아야 할까., 어떤 바이오 기업에 ..."
12,"파운트, 한화생명에 `AI 변액보험 펀드관리` 제공","카톡으로 변액보험 펀드 조회, 포트폴리오 추천 등 제공연금수령시기 다가오면 자동으로...",과학,"[카톡으로 변 액보험 펀드 조회, 포트폴리오 추천 등 제공연금 수령시기 다가오면 자..."
13,"박승원 광명시장, “업사이클을 넘어 에코디자인 문화 산업으로”","환경 보호 방법은 규제를 통한 억제가 아닌, 수요 주체의 능동적인 태도 변화로부터 ...",과학,"[환경 보호 방법은 규제를 통한 억제가 아닌, 수요 주체의 능동적인 태도 변화로부터..."
14,"원더피플, 배틀로얄 슈팅게임 ‘슈퍼피플’ 글로벌 테스트",게임 개발사 원더피플(대표 류제일)은 자체 개발해 서비스를 앞둔 슈팅 게임 ‘슈퍼...,과학,[게임 개발 사 원 더 피플( 대표 류 제일) 은 자체 개발해 서비스를 앞둔 슈팅 ...


In [5]:
df2['요약문'] = df2['s1'].apply(lambda x: smz.run(x))

----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 중입니다----------
----------뉴스 기사를 요약 

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
df2

,title,contents,분야,s1,요약문
0,"국민의힘 정책 알릴 플랫폼, 네이버도 카카오도 페이스북도 아닌, ‘나무위키’",“앞으로 모든 정책과 공약은 나무위키 통해서 할 것” 국민의힘이 내년 ...,정치,[“ 앞으로 모든 정책과 공약은 나무 위 키 통해서 할 것” 국민의 힘이 내년 대선...,“ 앞으로 모든 정책과 공약은 나무 위 키 통해서 할 것” 국민의 힘이 내년 대선을...
1,"하면 된다?…2030 지지율 역전 李, 하루걸러 하루 ‘청년표’ 노크",이재명 더불어민주당 대선 후보가 7일 오후 서울 마포구 한 모임 공간(앤드스페이...,정치,[이재명 더불어 민주당 대선 후보가 7일 오후 서울 마포구 한 모임 공간( 앤드 스...,이재명 더불어 민주당 대선 후보가 7일 오전 서울 관악구 서울대학교에서 열린 ' 청...
2,"윤석열 ""집권하면 북한 피격 공무원 수사정보 공개하겠다""","윤석열 ""집권하면 북한 피격 공무원 수사정보 공개하겠다"" 사진=연합뉴스국민의힘 윤석...",정치,"[윤석 열 "" 집권하면 북한 피격 공무원 수사정보 공개하겠다"" 사진= 연합 뉴스 국...",지난 11월 30일 동부 지검도 같은 결정을 내렸다며 국가기관이 정부의 공식 수사결...
3,"文대통령, 12일 호주 국빈방문…청와대 “방역 만전 기해”",문재인 대통령이 지난 10월 31일(현지시간) 이탈리아 로마 누볼라 컨벤션 양자회담...,정치,[문 재인 대통령이 지난 10월 31일( 현지시간) 이탈리아 로마 누 볼라 컨벤션 ...,문 재인 대통령이 지난 10월 31일( 현지시간) 이탈리아 로마 누 볼라 컨벤션 양...
4,"금태섭 의원에 묻고 싶다, 국민의힘은 뭐가 바뀌었나","""주 52시간 폐지? 윤석열, 노동관 바꾸지 않으면 사퇴해야"" http://omn...",정치,"["" 주 52 시간 폐지? 윤석 열, 노동 관 바꾸지 않으면 사퇴해야"" http: ...","본인의 소 셜 미디어에 세월 호 7 주기 행사를 두고 "" 차량행동인지 함께 걷기..."
...,...,...,...,...,...
11,“2022년 화두는 알츠하이머병·유전자치료제…신약개발사 관심”,투자 토크쇼바이오 기업에 투자하려면 무엇을 알아야 할까. 어떤 바이오 기업에 투자...,과학,"[투자 토크쇼 바이오 기업에 투자하려면 무엇을 알아야 할까., 어떤 바이오 기업에 ...",투자 토크쇼 바이오 기업에 투자하려면 무엇을 알아야 할까. 어떤 바이오 기업에 투자...
12,"파운트, 한화생명에 `AI 변액보험 펀드관리` 제공","카톡으로 변액보험 펀드 조회, 포트폴리오 추천 등 제공연금수령시기 다가오면 자동으로...",과학,"[카톡으로 변 액보험 펀드 조회, 포트폴리오 추천 등 제공연금 수령시기 다가오면 자...","카톡으로 변 액보험 펀드 조회, 포트폴리오 추천 등 제공연금 수령시기 다가오면 자동..."
13,"박승원 광명시장, “업사이클을 넘어 에코디자인 문화 산업으로”","환경 보호 방법은 규제를 통한 억제가 아닌, 수요 주체의 능동적인 태도 변화로부터 ...",과학,"[환경 보호 방법은 규제를 통한 억제가 아닌, 수요 주체의 능동적인 태도 변화로부터...",업 사이클링을 비롯한 친환경 제품 관련 시장은 환경 보호와 함께 경제적 실익이라는 ...
14,"원더피플, 배틀로얄 슈팅게임 ‘슈퍼피플’ 글로벌 테스트",게임 개발사 원더피플(대표 류제일)은 자체 개발해 서비스를 앞둔 슈팅 게임 ‘슈퍼...,과학,[게임 개발 사 원 더 피플( 대표 류 제일) 은 자체 개발해 서비스를 앞둔 슈팅 ...,게임 개발 사 원 더 피플( 대표 류 제일) 은 자체 개발해 서비스를 앞둔 슈팅 게...


In [58]:
result = smz.concatDF(df1,df2)

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
result

,title,contents,분야,요약문
0,"이재용 부회장, 중동행…UAE서 신사업 기회 모색",美 방문 후 2주만에 다시 해외로...재판 종료 직후 출장길글로벌 네트워크 복원 및...,경제,美 방문 후 2주 만에 다시 해외로... 재판 종료 직후 출장길 글로벌 네트워크 복...
1,"JP모건, 허츠 투자의견 '비중확대'…목표가 30달러",JP모건이 미국의 렌터카 업체 허츠 글로벌 홀딩스(Hertz Global Holdi...,경제,JP 모건이 미국의 렌터카 업체 허 츠 글로벌 홀딩 스 (Hertz Global H...
2,"이재용 부회장, 미국 이어 중동行…""9일 귀국 예정""",7일 밤 출국…UAE 등 중동 출장아부다비 왕세제 등 중동 주요인사 만날 듯북미 출...,경제,7일 밤 출국 …UAE 등 중동 출장 아부 다비 왕세제 등 중동 주요인사 만날 듯 ...
3,비트코인 흔들리자 P2E 게임株도 출렁…옥석가리기 본격화,주말 사이 비트코인 20% 넘게 하락하며 '출렁'국내 가상자산 거래소 등 관련株 줄...,경제,주말 사이 비트 코인 20% 넘게 하락하며 ' 출렁' 국내 가상자산 거래소 등 관련...
4,전남 영암 산란계 농장 AI 확진…에그플레이션 우려 '고조',산란계 3만6000마리 살처분…두 번째 발생 충남 천안에 이어 전남 영암의 산란계...,경제,산란계 3 만 6000마리 살처분… 두 번째 발생 충남 천안에 이어 전 남 ...
...,...,...,...,...
111,"與 ""'5·18 부정' 비니좌 노재승 활약상 일베에 버금가…윤석열 실언과 닮았다""","""공동선대위원장으로 영입된 노재승, 과거 자신의 SNS에 온갖 혐오 발언 쏟아내""노...",정치,""" 공동 선대위원장으로 영입된 노 재 승, 과거 자신의 SNS에 온갖 혐오 발언 쏟..."
112,"김종인 ""국민 생존 위협 느끼는데 무슨 자유주의 신봉인가""","기사내용 요약""소득 양극은 더 벌어져…생존 해결이 제1과제""""윤석열이 자유주의 심취...",정치,"기사내용 요약"" 소득 양극은 더 벌어져… 생존 해결이 제 1 과제"""" 윤석 열이 자..."
113,"김병준, '김건희 질문'에 ""잘 몰라, 권양숙 여사와도 얘기 잘 안했다""",김병준 윤석열 국민의힘 대선 후보 상임 선대위원장이 “김건희씨와는 얘기를 나눈 적...,정치,김 위원장은 김 씨에 대해 “ 저는 잘 모른다 ”며 “( 얘기를) 해 본 적이 전혀...
114,"이재명 “대체 왜 이러나, 정말 답답” 정부 소상공인정책 비판","소상공인, 자영업자 문제 등 민생 이슈에서 주도권을 갖고 현 정부와의 정책적 거...",정치,"이 후보는 이후 인터뷰에서 확실히 새로운 민주당 후보가 나온다면, 그것도 교체라고 ..."
